# Data Transformation Pipeline for Logistic Regression Model

**Source Notebook:** `yogi_takehome-test_fintech01_senior-data-analyst.ipynb`

This notebook contains all data transformation steps extracted from the original analysis notebook, organized into distinct sections for easy understanding and reproduction.

---

## Table of Contents

1. [Date Conversion & Temporal Feature Engineering](#1-date-conversion--temporal-feature-engineering)
2. [Loan Term Grouping](#2-loan-term-grouping)
3. [Loan Purpose Text Cleaning](#3-loan-purpose-text-cleaning)
4. [Loan Purpose Standardization (Keyword Matching)](#4-loan-purpose-standardization-keyword-matching)
5. [Loan Purpose Grouping (Top 5 + Others)](#5-loan-purpose-grouping-top-5--others)
6. [Employment Type Text Cleaning](#6-employment-type-text-cleaning)
7. [Model Feature Selection](#7-model-feature-selection)
8. [Quarter Feature Extraction & Month Column Removal](#8-quarter-feature-extraction--month-column-removal)
9. [One-Hot Encoding (Categorical Variables)](#9-one-hot-encoding-categorical-variables)
10. [Merge Application & Credit Features DataFrames](#10-merge-application--credit-features-dataframes)
11. [Filter Out Zero Credit Account Records](#11-filter-out-zero-credit-account-records)
12. [Drop Single-Value Column](#12-drop-single-value-column-all_timesincemostrecentdefault)
13. [Variance Inflation Factor (VIF) Analysis & Iterative Feature Removal](#13-variance-inflation-factor-vif-analysis--iterative-feature-removal)
14. [Outlier Removal (Z-Score Method)](#14-outlier-removal-z-score-method)
15. [Final Cleaned Dataset Summary](#15-final-cleaned-dataset-summary)

---

## Setup & Data Loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 100)
plt.style.use('seaborn-v0_8')

In [2]:
APPLICATION_PATH = '../data/application_samples.csv'
CREDIT_PATH = '../data/credit_features.csv'

application_df = pd.read_csv(APPLICATION_PATH)
credit_df = pd.read_csv(CREDIT_PATH)

print(application_df.shape)
print(credit_df.shape)

(8847, 7)
(8847, 14)


In [3]:
# Data transformation pipeline organized into 15 distinct sections below.
# Each section handles a specific transformation step for the logistic regression model.

---

## 1. Date Conversion & Temporal Feature Engineering

**Purpose:** Convert string dates to datetime format and extract temporal features (Month, Quarter) for time-based analysis.

**Columns Created:**
- `ApplicationDate` → converted to datetime
- `Month` → first day of the month (for grouping)
- `Quarter` → quarter string (e.g., "2020Q1")
- `SuccessLabel` → human-readable label ("Success" / "Fail")

In [4]:
def preprocess_applications(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and engineer base application fields."""
    df = df.copy()
    
    # Convert date string to datetime
    df['ApplicationDate'] = pd.to_datetime(df['ApplicationDate'], dayfirst=True)
    
    # Ensure numeric types for Amount and Term
    df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
    df['Term'] = pd.to_numeric(df['Term'], errors='coerce')
    
    # Extract Month (first day of month for grouping)
    df['Month'] = df['ApplicationDate'].dt.to_period('M').dt.to_timestamp()
    
    # Extract Quarter as string (e.g., "2020Q1")
    df['Quarter'] = df['ApplicationDate'].dt.to_period('Q').astype(str)
    
    # Create human-readable Success label
    df['SuccessLabel'] = df['Success'].map({1: 'Success', 0: 'Fail'})
    
    return df

application_df = preprocess_applications(application_df)
print(f"Shape after preprocessing: {application_df.shape}")
application_df[['ApplicationDate', 'Month', 'Quarter', 'Success', 'SuccessLabel']].head()

Shape after preprocessing: (8847, 10)


,ApplicationDate,Month,Quarter,Success,SuccessLabel
0,2020-07-03,2020-07-01,2020Q3,0,Fail
1,2020-02-04,2020-02-01,2020Q1,0,Fail
2,2020-08-02,2020-08-01,2020Q3,0,Fail
3,2020-09-23,2020-09-01,2020Q3,0,Fail
4,2020-01-01,2020-01-01,2020Q1,0,Fail


---

## 2. Loan Term Grouping

**Purpose:** Group individual loan terms into broader categories to reduce granularity and improve statistical power for less common terms.

**Mapping Logic:**
- 12 months → "12"
- 18, 24 months → "18-24"
- 30, 36 months → "30-36"
- 42, 48 months → "42-48"
- 54, 60 months → "54-60"

**Column Created:** `Term_grouped`

In [5]:
def group_terms(term):
    """Group loan terms into broader categories."""
    if term == 12:
        return '12'
    elif term in [18, 24]:
        return '18-24'
    elif term in [30, 36]:
        return '30-36'
    elif term in [42, 48]:
        return '42-48'
    elif term in [54, 60]:
        return '54-60'
    else:
        return str(term)

application_df['Term_grouped'] = application_df['Term'].apply(group_terms)

print("Term grouping distribution:")
print(application_df['Term_grouped'].value_counts().sort_index())

Term grouping distribution:
Term_grouped
12        875
18-24    1390
30-36    1594
42-48    2196
54-60    2792
Name: count, dtype: int64


---

## 3. Loan Purpose Text Cleaning

**Purpose:** Standardize raw loan purpose text by converting to lowercase and removing leading/trailing whitespace.

**Transformation:**
- Strip whitespace
- Convert to lowercase

**Column Created:** `LoanPurpose_clean`

In [6]:
# Step 1: Lowercase and strip whitespace
application_df["LoanPurpose_clean"] = (
    application_df["LoanPurpose"]
    .astype(str)
    .str.strip()
    .str.lower()
)

print(f"Unique raw loan purposes: {application_df['LoanPurpose'].nunique()}")
print(f"Unique cleaned loan purposes: {application_df['LoanPurpose_clean'].nunique()}")

Unique raw loan purposes: 2444
Unique cleaned loan purposes: 2041


---

## 4. Loan Purpose Standardization (Keyword Matching)

**Purpose:** Map similar loan purpose phrases to standardized categories using keyword matching to consolidate variations (e.g., "debt consolidation", "consolidate debts", "debt consolidate" → "debt consolidation").

**Keyword Mappings:**
- Contains "consolidat" or "debt" → "debt consolidation"
- Contains "home" AND "improv" → "home improvement"
- Otherwise → keep original cleaned value

**Column Created:** `LoanPurpose_standardized`

In [7]:
def standardize_loan_purpose(purpose):
    """Map loan purpose variations to standardized categories using keyword matching."""
    purpose = purpose.lower()
    
    # Debt consolidation variants
    if any(word in purpose for word in ['consolidat', 'debt']):
        return 'debt consolidation'
    
    # Home improvement variants
    if 'home' in purpose and 'improv' in purpose:
        return 'home improvement'
    
    # Keep as-is for unmatched
    return purpose

application_df["LoanPurpose_standardized"] = (
    application_df["LoanPurpose_clean"]
    .apply(standardize_loan_purpose)
)

print(f"Unique standardized loan purposes: {application_df['LoanPurpose_standardized'].nunique()}")
print("\nTop 10 standardized loan purposes:")
print(application_df['LoanPurpose_standardized'].value_counts().head(10))

Unique standardized loan purposes: 1674

Top 10 standardized loan purposes:
LoanPurpose_standardized
debt consolidation    2072
car                   1372
home improvement       890
other                  371
new car                149
furniture              117
car purchase            85
car repairs             77
short term loan         70
car loan                64
Name: count, dtype: int64


---

## 5. Loan Purpose Grouping (Top 5 + Others)

**Purpose:** Reduce cardinality by keeping only the top 5 most frequent loan purposes and grouping all others into "others" category.

**Logic:**
- Keep top 5 standardized loan purposes by frequency
- All other purposes → "others"

**Column Created:** `LoanPurpose_grouped`

In [8]:
# Get top 5 standardized loan purposes (excluding "other" if present)
std_top5 = (
    application_df[application_df["LoanPurpose_standardized"] != "other"]["LoanPurpose_standardized"]
        .value_counts()
        .head(5)
)
std_top5_names = std_top5.index.tolist()

# Create grouped column: top 5 + others
application_df["LoanPurpose_grouped"] = application_df["LoanPurpose_standardized"].apply(
    lambda x: x if x in std_top5_names else "others"
)

print("Loan Purpose Grouped Distribution:")
print(application_df['LoanPurpose_grouped'].value_counts())

Loan Purpose Grouped Distribution:
LoanPurpose_grouped
others                4247
debt consolidation    2072
car                   1372
home improvement       890
new car                149
furniture              117
Name: count, dtype: int64


---

## 7. Model Feature Selection

**Purpose:** Select relevant columns from application dataframe for the logistic regression model.

**Columns Selected:**
- `UID` - unique identifier for merging
- `Month` - temporal feature (will be converted to Quarter)
- `Amount` - loan amount (continuous)
- `Term_grouped` - grouped loan term (categorical)
- `EmploymentType` - employment type (categorical)
- `LoanPurpose_grouped` - grouped loan purpose (categorical)
- `Success` - target variable (binary)

In [9]:
# Select columns for modeling
application_model_df = application_df[['UID', 'Month', 'Amount', 'Term_grouped', 'EmploymentType', 'LoanPurpose_grouped', 'Success']]

print(f"Model DataFrame shape: {application_model_df.shape}")
print(f"\nColumns selected: {list(application_model_df.columns)}")

Model DataFrame shape: (8847, 7)

Columns selected: ['UID', 'Month', 'Amount', 'Term_grouped', 'EmploymentType', 'LoanPurpose_grouped', 'Success']


---

## 8. Quarter Feature Extraction & Month Column Removal

**Purpose:** Extract Quarter from Month column for model features, then drop the Month column (too granular for the model).

**Transformation:**
- Extract quarter number (1-4) from Month datetime
- Drop Month column

**Column Created:** `Quarter` (integer 1-4)

In [10]:
# Create Quarter column from Month (integer 1-4)
application_model_df["Quarter"] = application_model_df["Month"].dt.quarter

# Drop the Month column (too granular for model)
application_model_df = application_model_df.drop(columns=["Month"])

print(f"Shape after Quarter extraction: {application_model_df.shape}")
print(f"Columns: {list(application_model_df.columns)}")

Shape after Quarter extraction: (8847, 7)
Columns: ['UID', 'Amount', 'Term_grouped', 'EmploymentType', 'LoanPurpose_grouped', 'Success', 'Quarter']


/var/folders/hf/0mp7k_vd30z8kr8nkwwh037m0000gn/T/ipykernel_63058/3936759089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  application_model_df["Quarter"] = application_model_df["Month"].dt.quarter


---

## 9. One-Hot Encoding (Categorical Variables)

**Purpose:** Convert categorical variables into binary dummy variables for logistic regression. Uses `drop_first=True` to avoid the dummy variable trap (multicollinearity).

**Columns Encoded:**
- `Term_grouped` → Term_grouped_18-24, Term_grouped_30-36, etc.
- `EmploymentType` → EmploymentType_Employed - part time, EmploymentType_Retired, etc.
- `LoanPurpose_grouped` → LoanPurpose_grouped_debt consolidation, etc.
- `Quarter` → Quarter_2, Quarter_3, Quarter_4

**Note:** First category of each variable is dropped as reference (baseline)

In [11]:
# One-hot encode categorical variables
application_model_df = pd.get_dummies(
    application_model_df,
    columns=['Term_grouped', 'EmploymentType', 'LoanPurpose_grouped', 'Quarter'],
    drop_first=True,  # Avoid dummy variable trap (multicollinearity)
    dtype=int  # Use 0/1 integers instead of True/False
)

# View the new columns created
print("New columns after one-hot encoding:")
encoded_cols = [col for col in application_model_df.columns 
                if 'Term_' in col or 'EmploymentType_' in col 
                or 'LoanPurpose_grouped_' in col or 'Quarter_' in col]
print(encoded_cols)

print(f"\nDataFrame shape after encoding: {application_model_df.shape}")

New columns after one-hot encoding:
['Term_grouped_18-24', 'Term_grouped_30-36', 'Term_grouped_42-48', 'Term_grouped_54-60', 'EmploymentType_Employed - part time', 'EmploymentType_Retired', 'EmploymentType_Self employed', 'LoanPurpose_grouped_debt consolidation', 'LoanPurpose_grouped_furniture', 'LoanPurpose_grouped_home improvement', 'LoanPurpose_grouped_new car', 'LoanPurpose_grouped_others', 'Quarter_2', 'Quarter_3', 'Quarter_4']

DataFrame shape after encoding: (8847, 18)


---

## 10. Merge Application & Credit Features DataFrames

**Purpose:** Combine application features with credit bureau features on the unique identifier (UID) to create a comprehensive feature set.

**Join Type:** Inner join (only keep records that exist in both datasets)

**Columns Added from credit_df:**
- ALL_AgeOfOldestAccount, ALL_AgeOfYoungestAccount
- ALL_Count, ALL_CountActive, ALL_CountSettled
- ALL_CountClosedLast12Months, ALL_CountDefaultAccounts, ALL_CountOpenedLast12Months
- ALL_MeanAccountAge
- ALL_SumCurrentOutstandingBal, ALL_SumCurrentOutstandingBalExcMtg
- ALL_TimeSinceMostRecentDefault
- ALL_WorstPaymentStatusActiveAccounts

In [12]:
# Merge application features with credit bureau features
combined_model_features_df = pd.merge(
    application_model_df, 
    credit_df, 
    on='UID', 
    how='inner'
)

print(f"Shape after merge: {combined_model_features_df.shape}")
print(f"Columns: {list(combined_model_features_df.columns)}")

Shape after merge: (8847, 31)
Columns: ['UID', 'Amount', 'Success', 'Term_grouped_18-24', 'Term_grouped_30-36', 'Term_grouped_42-48', 'Term_grouped_54-60', 'EmploymentType_Employed - part time', 'EmploymentType_Retired', 'EmploymentType_Self employed', 'LoanPurpose_grouped_debt consolidation', 'LoanPurpose_grouped_furniture', 'LoanPurpose_grouped_home improvement', 'LoanPurpose_grouped_new car', 'LoanPurpose_grouped_others', 'Quarter_2', 'Quarter_3', 'Quarter_4', 'ALL_AgeOfOldestAccount', 'ALL_AgeOfYoungestAccount', 'ALL_Count', 'ALL_CountActive', 'ALL_CountClosedLast12Months', 'ALL_CountDefaultAccounts', 'ALL_CountOpenedLast12Months', 'ALL_CountSettled', 'ALL_MeanAccountAge', 'ALL_SumCurrentOutstandingBal', 'ALL_SumCurrentOutstandingBalExcMtg', 'ALL_TimeSinceMostRecentDefault', 'ALL_WorstPaymentStatusActiveAccounts']


---

## 11. Filter Out Zero Credit Account Records

**Purpose:** Remove applicants with no credit history (ALL_Count = 0) as they:
1. Have 100% rejection rate (perfect separation issue)
2. Violate logistic regression assumptions
3. Should be routed to alternative review process

**Filter Condition:** Keep rows where `ALL_Count != 0`

In [13]:
# Count records with zero credit accounts
zero_credit_count = (combined_model_features_df['ALL_Count'] == 0).sum()
print(f"Records with ALL_Count = 0: {zero_credit_count}")

# Filter out zero credit account records
combined_model_features_df = combined_model_features_df[combined_model_features_df['ALL_Count'] != 0]

print(f"Shape after filtering: {combined_model_features_df.shape}")

Records with ALL_Count = 0: 108
Shape after filtering: (8739, 31)


---

## 12. Drop Single-Value Column (ALL_TimeSinceMostRecentDefault)

**Purpose:** Remove columns that have only one unique value (constant), as they provide no predictive information.

**Column Dropped:** `ALL_TimeSinceMostRecentDefault`

**Reason:** After filtering out zero credit accounts, this column may have only one value (or very limited variance)

In [14]:
# Drop column with single value (no predictive power)
combined_model_features_df = combined_model_features_df.drop('ALL_TimeSinceMostRecentDefault', axis=1)

print(f"Shape after dropping single-value column: {combined_model_features_df.shape}")

Shape after dropping single-value column: (8739, 30)


---

## 13. Variance Inflation Factor (VIF) Analysis & Iterative Feature Removal

**Purpose:** Detect and remove multicollinear features using Variance Inflation Factor (VIF). High multicollinearity inflates standard errors and makes coefficient interpretation unreliable.

**VIF Interpretation:**
- VIF > 10: High multicollinearity - Remove
- VIF > 5: Moderate - Consider removing
- VIF ≤ 5: OK - Keep

**Process:**
1. Calculate VIF for all features
2. Remove feature with highest VIF if VIF > threshold
3. Repeat until all VIF ≤ threshold

**Threshold Used:** VIF > 7.5

In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(X, feature_cols):
    """Calculate VIF for all features."""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = feature_cols
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data.sort_values("VIF", ascending=False).reset_index(drop=True)

def vif_status(vif):
    """Return status label based on VIF value."""
    if vif > 10:
        return "❌ High - Remove"
    elif vif > 5:
        return "⚠️ Moderate - Consider"
    else:
        return "✅ OK"

# Get feature columns (exclude UID and Success)
feature_cols = [col for col in combined_model_features_df.columns if col not in ['UID', 'Success']]
X = combined_model_features_df[feature_cols].copy()

# Iterative VIF removal
VIF_THRESHOLD = 7.5
removed_features = []
iteration = 0

print("Iterative VIF Analysis - Removing One Feature at a Time")
print("=" * 80)
print(f"Threshold: VIF > {VIF_THRESHOLD}")
print(f"Starting features: {len(feature_cols)}")
print("=" * 80)

while True:
    iteration += 1
    current_features = [col for col in feature_cols if col not in removed_features]
    X_current = combined_model_features_df[current_features]
    
    vif_data = calculate_vif(X_current, current_features)
    max_vif = vif_data["VIF"].max()
    max_vif_feature = vif_data.loc[vif_data["VIF"].idxmax(), "Feature"]
    
    if max_vif <= VIF_THRESHOLD:
        print(f"\n✅ Iteration {iteration}: All VIF values now ≤ {VIF_THRESHOLD}")
        break
    
    print(f"\n🔄 Iteration {iteration}: Highest VIF = {max_vif:,.2f} ({max_vif_feature})")
    removed_features.append(max_vif_feature)
    print(f"   ❌ Removed: {max_vif_feature}")

# Final summary
print("\n" + "=" * 80)
print("📊 SUMMARY")
print("=" * 80)
print(f"Features removed ({len(removed_features)}):")
for i, feat in enumerate(removed_features, 1):
    print(f"   {i}. {feat}")

# Store cleaned feature list
feature_cols_cleaned = [col for col in feature_cols if col not in removed_features]
print(f"\nRemaining features: {len(feature_cols_cleaned)}")

Iterative VIF Analysis - Removing One Feature at a Time
Threshold: VIF > 7.5
Starting features: 28

🔄 Iteration 1: Highest VIF = 560,152.45 (ALL_Count)
   ❌ Removed: ALL_Count

🔄 Iteration 2: Highest VIF = 18.34 (ALL_MeanAccountAge)
   ❌ Removed: ALL_MeanAccountAge

🔄 Iteration 3: Highest VIF = 9.30 (ALL_CountActive)
   ❌ Removed: ALL_CountActive

✅ Iteration 4: All VIF values now ≤ 7.5

📊 SUMMARY
Features removed (3):
   1. ALL_Count
   2. ALL_MeanAccountAge
   3. ALL_CountActive

Remaining features: 25


---

## 14. Outlier Removal (Z-Score Method)

**Purpose:** Remove extreme outliers from monetary columns that could unduly influence the logistic regression model.

**Method:** Z-Score based removal
- Calculate mean and standard deviation
- Remove rows where |value - mean| > 3 × standard deviation
- This corresponds to approximately 0.3% of data in a normal distribution

**Column Targeted:** `ALL_SumCurrentOutstandingBal`

In [16]:
# Remove outliers where Z-score > 3 for ALL_SumCurrentOutstandingBal
col = 'ALL_SumCurrentOutstandingBal'
mean = combined_model_features_df[col].mean()
std = combined_model_features_df[col].std()

# Count before
rows_before = len(combined_model_features_df)

# Keep rows where Z-score <= 3
combined_model_features_df = combined_model_features_df[
    ((combined_model_features_df[col] - mean).abs() <= 3 * std)
]

# Count after
rows_after = len(combined_model_features_df)
rows_removed = rows_before - rows_after

print(f"Outlier Removal Summary ({col}):")
print(f"  Rows before: {rows_before}")
print(f"  Rows after: {rows_after}")
print(f"  Rows removed: {rows_removed} ({rows_removed/rows_before*100:.2f}%)")

Outlier Removal Summary (ALL_SumCurrentOutstandingBal):
  Rows before: 8739
  Rows after: 8621
  Rows removed: 118 (1.35%)


---

## 15. Final Cleaned Dataset Summary

**Purpose:** Display the final shape and columns of the cleaned dataset ready for logistic regression modeling.

In [17]:
# Final dataset summary
print("=" * 80)
print("FINAL CLEANED DATASET SUMMARY")
print("=" * 80)
print(f"Total rows: {len(combined_model_features_df)}")
print(f"Total columns: {len(combined_model_features_df.columns)}")
print(f"\nFeature columns (for modeling): {len(feature_cols_cleaned)}")
print(f"Target column: Success")
print(f"\nFeatures ready for Logistic Regression:")
for i, col in enumerate(feature_cols_cleaned, 1):
    print(f"  {i:2d}. {col}")

# Prepare final X and y for modeling
X_final = combined_model_features_df[feature_cols_cleaned]
y_final = combined_model_features_df['Success']

print(f"\n✅ X shape: {X_final.shape}")
print(f"✅ y shape: {y_final.shape}")

FINAL CLEANED DATASET SUMMARY
Total rows: 8621
Total columns: 30

Feature columns (for modeling): 25
Target column: Success

Features ready for Logistic Regression:
   1. Amount
   2. Term_grouped_18-24
   3. Term_grouped_30-36
   4. Term_grouped_42-48
   5. Term_grouped_54-60
   6. EmploymentType_Employed - part time
   7. EmploymentType_Retired
   8. EmploymentType_Self employed
   9. LoanPurpose_grouped_debt consolidation
  10. LoanPurpose_grouped_furniture
  11. LoanPurpose_grouped_home improvement
  12. LoanPurpose_grouped_new car
  13. LoanPurpose_grouped_others
  14. Quarter_2
  15. Quarter_3
  16. Quarter_4
  17. ALL_AgeOfOldestAccount
  18. ALL_AgeOfYoungestAccount
  19. ALL_CountClosedLast12Months
  20. ALL_CountDefaultAccounts
  21. ALL_CountOpenedLast12Months
  22. ALL_CountSettled
  23. ALL_SumCurrentOutstandingBal
  24. ALL_SumCurrentOutstandingBalExcMtg
  25. ALL_WorstPaymentStatusActiveAccounts

✅ X shape: (8621, 25)
✅ y shape: (8621,)
